In [ ]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images #Raju: change to "visualize"
import mrcnn.model as modellib
from mrcnn.model import log

import custom #Raju: for bridge change it to "custom" 

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

custom_WEIGHTS_PATH = "C:/Users/mk5n2/Mask_RCNN/logs/resnet50_train_1/mask_rcnn_roi_0060.h5"  # weight of train_1 (40imgs)
# custom_WEIGHTS_PATH = "C:/Users/mk5n2/logs/resnet50_train_2/mask_rcnn_roi_0100.h5"  # weight of 1st iteration (97imgs)
# custom_WEIGHTS_PATH = "C:/Users/mk5n2/logs/resnet50_train_1_supervised/mask_rcnn_roi_0100.h5" # supervised
# custom_WEIGHTS_PATH = "C:/Users/mk5n2/logs/supervised_train_1_small_data/mask_rcnn_roi_0100.h5" # supervised_small_data
# custom_WEIGHTS_PATH = "C:/Users/mk5n2/logs/roi20190401T1756/mask_rcnn_roi_0080.h5" # 80% + 20% semi + sup
# custom_WEIGHTS_PATH = "C:/Users/mk5n2/logs/roi20190330T1903/mask_rcnn_roi_0120.h5" # 2nd iteration (semi-supervised)(133 imgs)

In [ ]:
config = custom.CustomConfig() #Raju: for bridge change "custom_matc" to "custom"
custom_DIR = os.path.join(ROOT_DIR, "customImages")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
# Here, two parameters we should change
# 1. backbone: resnet 50, resnet101-- from config.py
# 2. Detection_Min_Confidence: 0.5/0.9,... ... --from config.py and custom.py
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Load validation dataset
dataset = custom.CustomDataset() #Raju: for bridge change "custom_matc" to "custom"
dataset.load_custom(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

### The following line is only for the detection code

In [ ]:
# My original detection code: (Without temporal coherence)
#-------------------------------------------------------------
# Test results will be saved in the following directory
testresult = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/testresult/precision_previous/'

# Images from the following directory will be tested
test_dir ='C:/Users/mk5n2/Mask/Inspire/CustomImages/test/test_dir/precision/'

# Creating a list with the names of all the test images from the test_dir
testfiles= os.listdir(test_dir)
testfiles = testfiles[4:6]

#looping through all images in the test_dir
for i in range(len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
#     plt.savefig(testresult+testfiles[i])  

    
print('done')

### Precision_recall only in Mask RCNN

In [ ]:
import winsound
from mrcnn import utils
# Test results will be saved in the following directory
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/testresult/precision_train_1/'

# Images from the following directory will be tested
test_dir = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/test_dir/precision_train_1/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[3:4]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor=[]
Ycor=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i, name in enumerate(testfiles):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config,
                           i, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    
    filename= os.path.join(test_dir,testfiles[i])
#     image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
#     ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    Xc= converted[1] # X coordinate of all objects will be stored 
    Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    gt_match, pred_match,overlaps =utils.compute_precision_recall_old_version(file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'])
#     visualize_frame_relation_4f.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset.class_names, r['scores'], ax=ax,
#                             title="Predictions")
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    print(class_ids, 'for i =', i)
    print(scores, 'for i =', i)
    print(filename)
      

frequency = 3500  # Set Frequency To 2500 Hertz
duration = 100  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

### Detection Code with temporal coherence

In [ ]:
#Detection Code with temporal coherence
import winsound
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/testresult/iteration_1_supervised/'

# Images from the following directory will be tested
test_dir ='C:/Users/mk5n2/Mask/Inspire/CustomImages/test/test_dir/10frames_bridgeC_part1/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[0:3]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
b=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor1=[]
Ycor1=[]
Xcor2=[]
Ycor2=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]

for i in range(0,len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    if converted == []:
        Xc = []
        Yc = []
    else:
        Xc = converted[1]
        Yc = converted[0]
#     Xc= converted[1] # X coordinate of all objects will be stored 
#     Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    print("---")
    print(i)
    print(file)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    
    
    if i in range(0,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f1'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        a= cl
        Xcor1 = Xco
        Ycor1 = Yco
        
        plt.savefig(testresult+testfiles[i])
        
        
    elif i in range(1,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f2'
        print(f_ref)        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        b= cl
        Xcor2 = Xco
        Ycor2 = Yco
        
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(2,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f3'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        c= cl
        Xcor3 = Xco
        Ycor3 = Yco
        
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(3,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[] 
        f_ref = 'f4'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco
       
        plt.savefig(testresult+testfiles[i])

    elif i in range(4,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f5'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_annotation(f_ref,image,file,filename,filesize, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
        
        plt.savefig(testresult+testfiles[i])

frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1200  # Set Duration To 1000 ms == 1 second
# winsound.Beep(frequency, duration)

In [ ]:
#Detection Code with temporal coherence
import winsound
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask_RCNN/CustomImages/test/test_result/0th_iteration/'

# Images from the following directory will be tested
test_dir ='C:/Users/mk5n2/Mask_RCNN/CustomImages/test/test_dir/10frames_bridgeC_part1/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[567:578]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
b=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor1=[]
Ycor1=[]
Xcor2=[]
Ycor2=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
json1 ={}
for i in range(0,len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    file = testfiles[i]
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    if converted == []:
        Xc = []
        Yc = []
    else:
        Xc = converted[1]
        Yc = converted[0]
#     Xc= converted[1] # X coordinate of all objects will be stored 
#     Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    print("---")
    print(i)
    print(file)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    
    
    if i in range(0,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f1'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)
        print('before iteration f1: ',cl)

        visualize_frame_relation_4f.display_instances_5f_for_json(f_ref,image,file,filename,filesize,json1, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        a= cl
        print('after iteration f1: ', a)
        Xcor1 = Xco
        Ycor1 = Yco
        
        plt.savefig(testresult+testfiles[i])
        
        
    elif i in range(1,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f2'
        print(f_ref)        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)
        print('before iteration f2: ',cl)

        visualize_frame_relation_4f.display_instances_5f_for_json(f_ref,image,file,filename,filesize,json1, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        b= cl
        print('after iteration f2: ', b)
        Xcor2 = Xco
        Ycor2 = Yco
        
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(2,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f3'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_json(f_ref,image,file,filename,filesize,json1, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        c= cl
        Xcor3 = Xco
        Ycor3 = Yco
        
        plt.savefig(testresult+testfiles[i])
        
    elif i in range(3,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[] 
        f_ref = 'f4'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_json(f_ref,image,file,filename,filesize,json1, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco
        
        plt.savefig(testresult+testfiles[i])

    elif i in range(4,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f5'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        visualize_frame_relation_4f.display_instances_5f_for_json(f_ref,image,file,filename,filesize,json1, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names,a,b,c,d,e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5, r['scores'], ax=ax,
                        title="Predictions")
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
        
        plt.savefig(testresult+testfiles[i])

frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1200  # Set Duration To 1000 ms == 1 second
# winsound.Beep(frequency, duration)

### Precision_recall with temporal coherence

In [ ]:
#precision_recall with temporal coherence
import winsound
from mrcnn import visualize_frame_relation_4f
testresult = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/testresult/precision_train_1/'

# Images from the following directory will be tested
# test_dir = os.path.join(ROOT_DIR, "CustomImages/test/test_dir/10frames_bridgeC_part1/")
test_dir = 'C:/Users/mk5n2/Mask/Inspire/CustomImages/test/test_dir/precision_train_1/'

# Creating a list with the names of all the test images in the test_dir
testfiles= os.listdir(test_dir)
# testfiles = testfiles[184:210]
# in the 'c' variable we will store class_ids after every three images
c=[]
a=[]
b=[]
d=[]
e=[]
# s will store prediction score of joint
s=[]
# ita will store every iteration number
ita=[]
Xcor1=[]
Ycor1=[]
Xcor2=[]
Ycor2=[]
Xcor3=[]
Ycor3=[]
Xcor4=[]
Ycor4=[]
Xcor5=[]
Ycor5=[]
for i in range(0,len(testfiles)):
    filename= os.path.join(test_dir,testfiles[i])
    image = skimage.io.imread(filename)
    file = testfiles[i]
    
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config,
                           i, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    filesize = os.path.getsize(filename)
    
    #Run Detection
    results = model.detect([image], verbose=1)
    
    #Display results
    ax = get_ax(1)
    r = results[0]
    class_ids = r['class_ids']
    scores = r['scores']
    
    roi = r['rois']
    t_roi = zip(*roi) #transpose roi
    converted = list(t_roi) # converted to list from zip
    if converted == []:
        Xc = []
        Yc = []
    else:
        Xc = converted[1]
        Yc = converted[0]
#     Xc= converted[1] # X coordinate of all objects will be stored 
#     Yc= converted[0] # Y coordinate of all objects will be stored
    
    mapped = zip(scores, class_ids,Xc,Yc)
    mapped = list(mapped)
    print("---")
    print(i)
    print(file)
    
    # new variable 'a' is defined
    # after every two images this 'a' variable will store the class_id for that image
    
    
    if i in range(0,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f1'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall_old_18_march(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        a= cl
        Xcor1 = Xco
        Ycor1 = Yco
        
        
    elif i in range(1,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f2'
        print(f_ref)        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall_old_18_march(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        b= cl
        Xcor2 = Xco
        Ycor2 = Yco
        
        
    elif i in range(2,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f3'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall_old_18_march(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        c= cl
        Xcor3 = Xco
        Ycor3 = Yco

        
    elif i in range(3,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[] 
        f_ref = 'f4'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall_old_18_march(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        d= cl
        Xcor4 = Xco
        Ycor4 = Yco


    elif i in range(4,len(testfiles),5):
        sc=[] #this will store scores
        cl=[] # this will store class_ids
        Xco =[]
        Yco =[]
        f_ref = 'f5'
        print(f_ref)
        
#       this loop will eliminate all scores below 0.9 and will save remaining class_ids in cl
        for k,l,m,n in mapped:
            if k<0.9:
                continue
            sc.append(k)
            cl.append(l)
            Xco.append(m)
            Yco.append(n)

        gt_match, pred_match,overlaps =utils.compute_precision_recall_old_18_march(f_ref, file,gt_bbox, gt_class_id, gt_mask,
                     r["rois"], r["class_ids"], r["scores"], r['masks'],a,b, c, d, e,cl,Xco,Yco,Xcor1,Ycor1,Xcor2,Ycor2,Xcor3,Ycor3,Xcor4,Ycor4,Xcor5,Ycor5)
        e= cl
        Xcor5 = Xco
        Ycor5 = Yco
        

frequency = 3500  # Set Frequency To 2500 Hertz
duration = 1500  # Set Duration To 1000 ms == 1 second
# winsound.Beep(frequency, duration)

### mAP calculation in mask rcnn without temporal coherence

In [ ]:
from mrcnn import utils
#mAP calculation:
image_ids = os.listdir('C:/Users/mk5n2/Mask/Inspire/CustomImages/test/test_dir/pre_recall_curve/')
# image_ids = image_ids[58:60]
image_ids
APs =[]

for image_id, name in enumerate(image_ids):
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, config), 0)
    # Run object detection

    results = model.detect([image], verbose=0)
    ax = get_ax(1)
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
    
#     # Compute AP over range 0.5 to 0.95 and print it
#     utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
#                            r['rois'], r['class_ids'], r['scores'], r['masks'],
#                            verbose=1)

# #     visualize.display_differences(
# #         image,
# #         gt_bbox, gt_class_id, gt_mask,
# #         r['rois'], r['class_ids'], r['scores'], r['masks'],
# #         dataset.class_names, ax=get_ax(),
# #         show_box=False, show_mask=False,
# #         iou_threshold=0.5, score_threshold=0.5)
#     # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
#     print("AP : ",AP , "for image id: ", image_id)

# #     print("precisions :", precisions)
# #     print("recalls :", recalls)
# #     print("overlaps :", overlaps)
#     APs.append(AP)
    
# print("mAP: ", np.mean(APs))
# print (APs)

In [ ]:
# Draw precision-recall curve
AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)
print('precisions: ',precisions)
print('recalls: ',recalls)
print(AP)

In [ ]:
# Grid of ground truth objects and their predictions
visualize_frame_relation_4f.plot_overlaps(gt_class_id, r['class_ids'], r['scores'],
                        overlaps, dataset.class_names)

### Below code are obsolete (18 March,2019)